In [14]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet50, resnet152
from PIL import Image
import numpy as np
import os
import requests
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib import colormaps

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Step 1: Load Pre-trained ResNet18 Model
model = resnet18(pretrained=True).to(device)
model.eval()  # Set to evaluation mode

# Step 2: Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 as required by ResNet18
    transforms.ToTensor(),          # Convert image to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Step 3: Load ImageNet Class Labels
imagenet_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_labels = requests.get(imagenet_url).json()

# Step 4: Grad-CAM Helper Functions
activation_maps = {}
gradients = {}

# Hook to capture activation maps
def save_activation(name):
    def hook(module, input, output):
        activation_maps[name] = output.detach()
    return hook

# Hook to capture gradients
def save_gradient(name):
    def hook(module, grad_in, grad_out):
        gradients[name] = grad_out[0]
    return hook

# Register hooks for all convolutional layers
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        module.register_forward_hook(save_activation(name))
        module.register_backward_hook(save_gradient(name))

# Function to compute Grad-CAM
def generate_gradcam(activation, gradient):
    weights = gradient.mean(dim=(2, 3), keepdim=True)  # Global average pooling of gradients
    cam = (weights * activation).sum(dim=1, keepdim=True)  # Weighted sum of activations
    cam = F.relu(cam)  # Apply ReLU
    cam = cam[0, 0].cpu().numpy()  # Remove batch and channel dimensions
    
    # Normalize to [0, 1] with safeguards
    cam_min, cam_max = cam.min(), cam.max()
    if cam_max - cam_min == 0:
        cam = np.zeros_like(cam)  # If max equals min, return a zero array
    else:
        cam = (cam - cam_min) / (cam_max - cam_min)
    
    return cam

# Function to save Grad-CAM heatmap with colormap
def save_gradcam(cam, image_size, output_path):
    # Apply colormap
    cmap = colormaps.get_cmap('jet')  # Updated method for fetching colormap
    cam_colored = cmap(cam)  # Apply colormap (RGBA)
    cam_colored = np.uint8(cam_colored[:, :, :3] * 255)  # Convert to RGB
    
    # Resize to original image size using LANCZOS for better quality
    cam_resized = Image.fromarray(cam_colored).resize(image_size, Image.LANCZOS)
    cam_resized.save(output_path)

# Step 5: Predict with Grad-CAM and Save Outputs
def predict_with_gradcam(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Forward pass
    outputs = model(input_tensor)
    probabilities = F.softmax(outputs[0], dim=0)
    confidence, predicted_idx = torch.max(probabilities, 0)
    predicted_class = imagenet_labels[predicted_idx.item()]

    # Backward pass
    model.zero_grad()
    outputs[0, predicted_idx].backward(retain_graph=True)

    # Create output directory
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_dir = os.path.join("cnn_images", base_name)
    os.makedirs(output_dir, exist_ok=True)

    # Save Grad-CAM for each convolutional layer
    print(f"Image: {image_path}, Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")
    for idx, (layer_name, activation) in enumerate(activation_maps.items(), start=1):
        gradient = gradients[layer_name]
        cam = generate_gradcam(activation, gradient)

        # Save the Grad-CAM heatmap with colormap
        output_path = os.path.join(output_dir, f"cnn_layer_{idx}.png")
        # save_gradcam(cam, image.size, output_path)

# Step 6: Test with Local Images
test_images = [
    "new_test_img/abacus.jpg",
    "new_test_img/arctic_fox.jpg",
    "new_test_img/chainsaw.jpg",
    "new_test_img/polaroid.jpg",
    "new_test_img/bottle.jpg",
]

for image_path in test_images:
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        continue
    predict_with_gradcam(image_path)


Using device: cpu
File not found: new_test_img/abacus.jpg
File not found: new_test_img/arctic_fox.jpg
File not found: new_test_img/chainsaw.jpg
File not found: new_test_img/polaroid.jpg
File not found: new_test_img/bottle.jpg
